In [ ]:
#hide
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
# default_exp perturb

# Perturb

> Functions for perturbing a graph 

## Edge deletions

In [ ]:
#export 
from nbdev.showdoc import *
import networkx as nx
import numpy as np
import pandas as pd
from grapht.graphtools import non_pendant_edges, has_isolated_nodes
from grapht.sampling import khop_subgraph, sample_edges

In [ ]:
#export 
def khop_remove(G, k, r, max_iter=100):
    """Removes r edges which are in a k-hop neighbourhood of some node, the perturbed graph will not have isolated nodes.
    
    If k is None then the samples are taken uniformly.
    
    This operation is attempted `max_iter` times, if no solution is found then `None` is returned.
    
    Returns:
        solution: a perturbed graph
        edges: a list of edges which were removed
        node: the node which the k-hop neighbourhood was taken around
    """
    solution = None
    for _ in range(max_iter):
        if k is not None:
            subgraph, node = khop_subgraph(G, k) 
        else:
            subgraph, node = G, None
        if len(non_pendant_edges(subgraph)) < r:
            continue
        edges = sample_edges(subgraph, r, non_pendant=True)
        Gp = G.copy()
        Gp.remove_edges_from(edges)
        if not has_isolated_nodes(Gp):
            solution = Gp
            break
    if solution is None:
        return None
    else:
        edge_info = pd.DataFrame(edges, columns=['u', 'v'])
        edge_info['type'] = 'remove'
        return solution, edge_info, node

## Rewiring

In [ ]:
#export
def khop_rewire(G, k, r, max_iter=100):
    """Rewire the graph in place where edges which are rewired are in a k-hop neighbourhood.
    
    A random k-hop neighbourhood is selected in G and r edges are rewired. 
    
    If the graph contains an isolated node this procedure is repeated.
    
    If `max_iter` attempted do not give a graph without isolated nodes `None` is returned.
    
    Returns:
        solution (nx.Graph): the rewired graph
        rewire_info (pd.DataFrame): a dataframe describing which edges were added or removed
        node: The node from which the k-hop neighbourhood was taken around
    """
    solution = None
    for _ in range(max_iter):
        if k is not None:
            subgraph, node = khop_subgraph(G, k)
        else:
            subgraph, node = G, None
        if len(subgraph.edges()) < r:
            continue
        edges = sample_edges(subgraph, r, non_pendant=False)
        Gp = G.copy()
        rewire_info = rewire(Gp, edges)
        if not has_isolated_nodes(Gp):
            solution = Gp
    if solution is None:
        return None
    else:
        return solution, rewire_info, node 
        
def rewire(G, edges):
    """Rewires `edges` in `G` inplace and returns a dataframe with the edges which were added or removed. 
    
    All edges are broken into stubs and then stubs are randomly joined together.
    
    Self loops are removed after the rewiring step.
    
    A dataframe is returned where each row is (u, v, 'add') or (u, v, 'remove'). 
    
    The dataframe will include entries (u, u, 'add') if self loops were added but these won't appear in the graph.
    """
    edges = np.array(edges)
    new_edges = np.reshape(np.random.permutation(edges.flatten()), (-1, 2))
    G.remove_edges_from(edges.tolist())
    G.add_edges_from(new_edges.tolist())
    G.remove_edges_from(nx.selfloop_edges(G))
    df_remove = pd.DataFrame(edges, columns = ['u', 'v'])
    df_remove['type'] = 'remove'
    df_add = pd.DataFrame(new_edges, columns = ['u', 'v'])
    df_add['type'] = 'add'
    return pd.concat([df_remove, df_add], ignore_index=True)

In [ ]:
#hide
from nbdev.export import notebook2script
notebook2script()

Converted 00_graphtools.ipynb.
Converted 01_sampling.ipynb.
Converted 02_metrics.ipynb.
Converted 03_perturb.ipynb.
Converted 04_plotting.ipynb.
Converted 05_data.ipynb.
Converted index.ipynb.
